In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import spacy as sp
import regex as re
    
from tensorflow import keras

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv("./train.csv")

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
SYMBOL_FILTER = re.compile("[!@#%:;,\.?\'\"]")
CLEAN_UP = re.compile("[^a-z]")
WHITESPCAE_FILTER = re.compile("  *")

def extract_words(x):
    x = " ".join(SYMBOL_FILTER.sub("",x.lower()).split())
    x = " ".join(CLEAN_UP.sub(" ",x).split())
    x = " ".join(WHITESPCAE_FILTER.sub(" ",x).split())
    return x

df['comment_text_clean'] = df.comment_text.apply(extract_words)

In [5]:
### Counting NUmber Of Words

from collections import Counter

In [6]:
words = Counter()

for line in df.comment_text_clean:
    for word in line.split():
        words[word] += 1
        

In [7]:
len(words.keys())

197406

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer()

In [10]:
features = cv.fit_transform(df.comment_text_clean.values.reshape(-1))

In [11]:
labels = df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].copy()

In [12]:
labels

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


### First Let's build a Toxic Comments Classifier

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score

In [14]:
X,x,Y,y = train_test_split(features,labels.toxic.values)

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
log = LogisticRegression()

In [17]:
log.fit(X,Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
y_pred = log.predict(x)

In [19]:
accuracy_score(y,y_pred),precision_score(y,y_pred),recall_score(y,y_pred)

(0.9470834482239991, 0.9005449591280654, 0.5117419354838709)

In [20]:
from tensorflow.keras import layers,optimizers,activations,losses,Sequential,utils

In [21]:
model = Sequential()

In [26]:
model = Sequential([
    layers.Dense(32, input_shape=(features.shape[1],)),
    layers.Activation('relu'),
    layers.Dense(2),
    layers.Activation('softmax'),
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 200

for epoch in range(15):
    print (epoch)
    for i in range(batch_size,X.shape[0],batch_size):
        x_batch = X[i-batch_size:i].toarray()
        y_batch = Y[i-batch_size:i]
        model.fit(x_batch,y_batch,batch_size=batch_size,verbose=2)
        
    x_batch = X[i:].toarray()
    y_batch = Y[i:]
    model.fit(x_batch,y_batch)

0
Train on 200 samples
200/200 - 0s - loss: 0.6886 - acc: 0.5700
Train on 200 samples
200/200 - 0s - loss: 0.6324 - acc: 0.9000
Train on 200 samples
200/200 - 0s - loss: 0.5419 - acc: 0.9200
Train on 200 samples
200/200 - 0s - loss: 0.5603 - acc: 0.8800
Train on 200 samples
200/200 - 0s - loss: 0.4832 - acc: 0.9150
Train on 200 samples
200/200 - 0s - loss: 0.4475 - acc: 0.9150
Train on 200 samples
200/200 - 0s - loss: 0.3950 - acc: 0.9350
Train on 200 samples
200/200 - 0s - loss: 0.4702 - acc: 0.9000
Train on 200 samples
200/200 - 0s - loss: 0.4415 - acc: 0.8650
Train on 200 samples
200/200 - 0s - loss: 0.3789 - acc: 0.9200
Train on 200 samples
200/200 - 0s - loss: 0.3647 - acc: 0.9200
Train on 200 samples
200/200 - 0s - loss: 0.4839 - acc: 0.9050
Train on 200 samples
200/200 - 0s - loss: 0.3624 - acc: 0.9150
Train on 200 samples
200/200 - 0s - loss: 0.4235 - acc: 0.9000
Train on 200 samples
200/200 - 0s - loss: 0.5180 - acc: 0.8850
Train on 200 samples
200/200 - 0s - loss: 0.3332 - ac

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Train on 200 samples
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-dc16a21d7e34>", line 19, in <module>
    model.fit(x_batch,y_batch,batch_size=batch_size,verbose=2)
  File "/usr/lib/python3/dist-packages/tensorflow_core/python/keras/engine/training.py", line 727, in fit
    use_multiprocessing=use_multiprocessing)
  File "/usr/lib/python3/dist-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 675, in fit
    steps_name='steps_per_epoch')
  File "/usr/lib/python3/dist-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 394, in model_iteration
    batch_outs = f(ins_batch)
  File "/usr/lib/python3/dist-packages/tensorflow_core/python/keras/backend.py", line 3461, in __call__
    dtype=tensor_type.as_numpy_dtype))
  File "/usr/local/lib/python3.6/dist-packages/numpy/core/_asar

KeyboardInterrupt: 

In [231]:
y_pred = []

for i in range(5000,x.shape[0],5000):
    y_pred += model.predict_classes(x[i-5000:i].toarray()).tolist()
    
y_pred += model.predict_classes(x[i:].toarray()).tolist()

In [232]:
accuracy_score(y_pred,y),recall_score(y_pred,y),precision_score(y_pred,y)

(0.9512445792494924, 0.8098846787479407, 0.6424464192368008)

### Using Tensoflow word embedding

In [22]:
# import tensorflow_hub as hub
# embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

# hub_layer = hub.KerasLayer(embedding, input_shape=[], 
#                            dtype=tf.string, trainable=True)
# # hub_layer(train_examples_batch[:3])